# Notebook 5: Composing Examples

In this notebook we will examine how we can use the elements we have encountered so far, in order to construct examples which will allow us to train machine learning models with data generated in real time. This is a core functionality of GravyFlow.

As usual, we begin by performing the relevent imports:

In [1]:
# Built-in imports
from typing import List, Dict, Iterator
from pathlib import Path

# Import the GravyFlow module.
import gravyflow as gf

# Dependency imports: 
import jax
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

## Creating an example generator through composition

We can combine all the elements we have seen so, noise generation, waveform generation, and waveform projection, and use MLy to create a custom Python generator.

In order to scale the injection with respect to the noise we can use a `gf.ScalingMethod` object. GravyFlow supports scaling with SNR (`gf.ScalingTypes.SNR`), HRSS (`gf.ScalingTypes.HRSS`), and HPEAK (`gf.ScalingTypes.HPEAK`).

- `value` : `Union[gf.Distribution, np.ndarray]`, Required
  > The value or distribution to use to scale the injections, units vary depending on type parameter.

- `type_` : `gf.ScalingTypes`, Required
  > Type of scaling, one of either `gf.ScalingTypes.SNR`, `gf.ScalingTypes.HRSS`, or `gf.ScalingTypes.HPEAK`.

Let us create a `gf.ScalingMethod` object:

In [2]:
# Create a scaling method object in order to scale the injection to the noise:
scaling_method : gf.ScalingMethod = gf.ScalingMethod(
    value=gf.Distribution(
        value=20,
        type_=gf.DistributionType.CONSTANT
    ),
    type_=gf.ScalingTypes.SNR
)

Next, we can set up all the other elements that we can use to compose or example generator:

In [3]:
# This object will be used to obtain real interferometer data based on specified parameters.
ifo_data_obtainer : gf.IFODataObtainer = gf.IFODataObtainer(
    observing_runs=gf.ObservingRun.O3, # Specify the observing run (e.g., O3).
    data_quality=gf.DataQuality.BEST,  # Choose the quality of the data (e.g., BEST).
    data_labels=[                      # Define the types of data to include.
        gf.DataLabel.NOISE, 
        gf.DataLabel.GLITCHES
    ],
    segment_order=gf.SegmentOrder.RANDOM, # Order of segment retrieval (e.g., RANDOM).
    force_acquisition=True,               # Force the acquisition of new data.
    cache_segments=False                  # Choose not to cache the segments.
)

# Initialize the noise generator wrapper:
# This wrapper will use the ifo_data_obtainer to generate real noise based on the specified parameters.
noise: gf.NoiseObtainer = gf.NoiseObtainer(
    ifo_data_obtainer=ifo_data_obtainer, # Use the previously set up IFODataObtainer object.
    noise_type=gf.NoiseType.REAL,        # Specify the type of noise as REAL.
    ifos=[gf.IFO.L1, gf.IFO.H1] # Specify the interferometer (e.g., LIGO Livingston L1).
)

# Define a uniform distribution for the mass of the first object in solar masses.
mass_1_distribution_msun : gf.Distribution = gf.Distribution(
    min_=5.0, 
    max_=90.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the mass of the second object in solar masses.
mass_2_distribution_msun : gf.Distribution = gf.Distribution(
    min_=5.0, 
    max_=90.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the inclination of the binary system in radians.
inclination_distribution_radians : gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=np.pi, 
    type_=gf.DistributionType.UNIFORM
)
    
# Initialize a CBCGenerator with the defined distributions and specific approximant.
# Available approximants: "IMRPhenomD", "IMRPhenomXAS", "IMRPhenomPv2", "TaylorF2", "IMRPhenomD_NRTidalv2"
cbc_generator : gf.WaveformGenerator = gf.CBCGenerator(
    approximant="IMRPhenomD", # Specify the waveform approximant here
    mass_1_msun=mass_1_distribution_msun,
    mass_2_msun=mass_2_distribution_msun,
    inclination_radians=inclination_distribution_radians,
    scaling_method=scaling_method,
    injection_chance=0.5,
    # For IMRPhenomPv2, you might want to specify spins if not using defaults:
    # spin_1_in=(0.0, 0.0, 0.0), 
    # spin_2_in=(0.0, 0.0, 0.0)
)
    
generator : Iterator = gf.data(       
    noise_obtainer=noise,
    waveform_generators=cbc_generator, # Use the ripple generator
    num_examples_per_batch=8,
    input_variables=[
        gf.ReturnVariables.WHITENED_ONSOURCE, 
    ],
    output_variables=[
        gf.ReturnVariables.INJECTIONS, 
        gf.ReturnVariables.WHITENED_INJECTIONS,
        gf.ReturnVariables.INJECTION_MASKS
    ]
)

2025-12-14 02:58:07,683 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:58:08,263 - INFO - Loading event times from cache.
2025-12-14 02:58:08,301 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:58:08,953 - INFO - Loading event times from cache.


Similiarly to the individual elements, we can use this example generator as an iterator, and produce N examples for use to examine:

In [4]:
# Generate a batch of examples using the composed generator.
input_data, output_data = next(generator)

2025-12-14 02:58:10,857 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1249712128.1, 1249713613.9), ext='gwf', urltype='file', match=None
2025-12-14 02:58:10,866 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:58:12,781 - INFO - Finding URLs for H-H1_HOFT_C01 in interval [1249712128.1, 1249713901.9), ext='gwf', urltype='file', match=None
2025-12-14 02:58:12,788 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025-12-14 02:58:14,431 - INFO - Finding URLs for L-L1_HOFT_C01 in interval [1244168826.1, 1244169965.9), ext='gwf', urltype='file', match=None
2025-12-14 02:58:14,446 - INFO - Validating SciToken with jti: https://cilogon.org/oauth2/6876e906ed8bece94f384a05891f9063?type=accessToken&ts=1765698359548&version=v2.0&lifetime=10800000
2025

We can print the output of the generator to examine its format, which values are returned will depend on which parameters we have requested in the input_variables, and output_variables field in our gf.data function. Both are returned in the form of a dictionary, which can easly be fed into a keras model if the inputs of the model are named similarly to the variable fields, we will show an example of this later in the notebook.

In [5]:
# This is the data we will uses as an input examples to out model:
print(f"Dictionary to feed the model: \n {input_data}")

# This is the target data we will use as labels to train our model:
print(f"Dictionary to use as the model labels: \n {output_data}")

Dictionary to feed the model: 
 {'WHITENED_ONSOURCE': Array([[[ 0.2778 ,  0.3696 , -0.3433 , ..., -0.2252 ,
          1.613  , -0.2354 ],
        [ 1.373  , -2.197  , -0.03955, ...,  1.257  ,
         -0.63   , -0.064  ]],

       [[ 0.9727 , -0.07446, -0.6387 , ..., -0.2249 ,
         -0.4187 ,  0.2869 ],
        [ 1.333  ,  1.125  ,  2.014  , ...,  0.1425 ,
          0.615  ,  1.03   ]],

       [[-0.2289 , -0.01346, -0.4766 , ...,  0.975  ,
         -0.5664 ,  1.099  ],
        [-1.539  ,  0.314  ,  0.3184 , ..., -0.2822 ,
         -0.10583, -0.1115 ]],

       ...,

       [[-0.4253 , -0.967  ,  1.626  , ...,  0.01423,
          0.6133 ,  0.042  ],
        [-2.438  ,  0.07635, -0.4373 , ..., -0.4673 ,
          0.3562 , -1.115  ]],

       [[ 0.7563 ,  0.3582 ,  0.3804 , ..., -0.1908 ,
          1.553  , -2.277  ],
        [ 0.382  , -0.7534 ,  0.4463 , ..., -0.858  ,
          1.187  , -2.219  ]],

       [[-0.6045 ,  0.6577 ,  0.2456 , ...,  1.594  ,
         -1.705  , -0.9937 ],

We can then print some examples from this dataset to examine the output:

In [6]:
# Extract the data from the generator output: 
onsource: jax.Array = input_data[gf.ReturnVariables.WHITENED_ONSOURCE.name]
injections: jax.Array = output_data[gf.ReturnVariables.INJECTIONS.name][0]
whitened_injections: jax.Array = output_data[gf.ReturnVariables.WHITENED_INJECTIONS.name][0]
injection_masks: jax.Array = output_data[gf.ReturnVariables.INJECTION_MASKS.name][0]

# Initialize an empty layout for the strain plots.
generated_data_layout: List = []


# Iterate over the multi-injections and their corresponding parameters.
for onsource_, injection, whitened_injection, masks_ in zip(
        onsource,
        injections,
        whitened_injections,
        injection_masks
    ):
    
    # Create strain plots for each Phenom D and WNB injection with titles displaying the parameter values.
    generated_data_layout.append([
        gf.generate_strain_plot(
            {
                "Onsouce": onsource_,
                "Whitened Injection": whitened_injection,
                "Injection": injection,
            },
            height=400,
            width=800,
            title=f"Example Output. Has Injection: {bool(masks_)}"
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(generated_data_layout)
output_notebook()
show(grid)

Loading BokehJS ...